# 視覺化進出場策略 (Visualizing Strategies)

## 看一下單一股票的進出場狀況

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import backtest

## 讀取歷史股價

In [ ]:
start = "2017-01-01"
end = "2017-12-31"

In [ ]:
# 讀取從指定日期之後的股價資訊
df = web.DataReader("TSLA", 'yahoo', start, end)

## 訂定策略

In [ ]:
# 突破策略
def breakout(df):
    # Donchian Channel
    df['20d_high'] = pd.Series.rolling(df['Adj Close'], window=20).max()
    df['10d_low'] = pd.Series.rolling(df['Adj Close'], window=10).min()

    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        # 進場條件
        if df['Adj Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        # 出場條件
        elif df['Adj Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()

In [ ]:
def macross(df):
    # 均線
    df['20d'] = pd.Series.rolling(df['Adj Close'], window=20).mean()
    df['60d'] = pd.Series.rolling(df['Adj Close'], window=60).mean()
    
    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        # 進場條件
        if df['20d'][t] > df['60d'][t] and df['20d'][t-1] < df['60d'][t-1] and df['60d'][t] > df['60d'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        # 出場條件
        elif df['Adj Close'][t] < df['60d'][t] and df['Adj Close'][t-1] < df['60d'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()

In [ ]:
def MyStrategy(df):
    # 均線
    df['5d'] = pd.Series.rolling(df['Adj Close'], window=5).mean()
    df['45d'] = pd.Series.rolling(df['Adj Close'], window=45).mean()
    
    has_position = False
    df['signals'] = 0
    for t in range(1, df['signals'].size):
        # 進場條件
        if df['5d'][t] > df['45d'][t] and df['5d'][t-1] < df['45d'][t-1] and df['45d'][t] > df['45d'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1
                has_position = True
        # 出場條件
        elif df['Adj Close'][t] < df['45d'][t] and df['Adj Close'][t-1] < df['45d'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1
                has_position = False

    df['positions'] = df['signals'].cumsum().shift()

In [ ]:
strategy = macross
strategy(df)

In [ ]:
# 底下這一行只是為了要在下面把 signals 跟 positions 畫出來做說明用
df[['signals', 'positions']].plot(subplots = True, ylim=(-1.1, 1.1), figsize = (10, 8))

## 標出進出場點

In [ ]:
fig, ax = plt.subplots()

ax.set_ylabel('Price')

lines = ['60d', '20d']

df['Adj Close'].plot(ax=ax, color='gray', lw=1., figsize=(10,8))
for i in lines:
    df[i].plot(ax=ax, lw=1., label=i)

# Plot the "buy" trades
ax.plot(df.loc[df.signals == 1].index,df['Adj Close'][df.signals == 1],'^', markersize=10, color='r', label='buy')

# Plot the "sell" trades
ax.plot(df.loc[df.signals == -1].index, df['Adj Close'][df.signals == -1], 'v', markersize=10, color='k', label='sell')

plt.legend()

## 計算Sharpe Ratio

In [ ]:
dailyRet = df['Adj Close'].pct_change()

In [ ]:
#假設無風險利率為 4%
#假設一年有252個交易日
excessRet = np.where(df['positions']==1, dailyRet, 0) - 0.04/252

sharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)

In [ ]:
sharpeRatio

## 計算MaxDD跟MaxDDD

In [ ]:
df['Ret'] = np.where(df['positions']==1, dailyRet, 0)
cumRet = np.cumprod(1 + df['Ret'])

In [ ]:
cumRet.plot(style='r-')

In [ ]:
backtest.DrawDownAnalysis(cumRet)